In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("output.csv")

In [3]:
df.head()

,binome_id,binome_statut,binome_date_proposition,binome_date_creation,binome_acceptance_delay,binome_date_update_statut,binome_cancellation_reason,binome_score,field_of_study,study_level,...,registration_date_x,desired_exchange_frequency,hobby,project_confidence_level,project_development_level,registration_date_y,workfield,current_role,needs_to_address,status_flag
0,157657,REJECTED,2024-08-01 14:54:30,NaN,0,2024-08-03 15:00:00,Not specified (or Not provided),7,NaN,NaN,...,NaN,Not specified (or Not provided),Not specified (or Not provided),3.0,3.0,2024-08-01 00:41:51,"Accounting, management",Senior Expertise Consulting & Auditing,"discover, to apply, motivate, perform",0
1,157701,REJECTED,2024-08-06 15:36:55,NaN,0,2024-08-08 16:00:00,Not specified (or Not provided),12,NaN,NaN,...,NaN,Not specified (or Not provided),Not specified (or Not provided),3.0,3.0,2024-08-01 14:40:09,Banking-Finance,Manager - Financial Evaluation,"discover, to apply, perform, english",0
2,157710,COMPLETED,2024-08-12 15:54:34,2024-08-12 17:51:39,117,2025-05-16 16:19:05,Not applicable,11,NaN,NaN,...,NaN,Not specified (or Not provided),Not specified (or Not provided),3.0,3.0,2024-08-11 03:17:54,Computer science,Network Engineer,"discover, to apply, motivate, perform, english",1
3,157715,CANCELLED,2024-08-12 16:54:52,2024-08-13 19:01:38,1566,2024-10-07 15:16:03,No response from the mentee,9,NaN,NaN,...,NaN,Not specified (or Not provided),Not specified (or Not provided),3.0,3.0,2024-08-08 15:37:28,Marketing-advertising,CRM Animation Manager,to apply,0
4,157729,REJECTED,2024-08-12 17:16:43,NaN,0,2024-08-14 18:00:02,Not specified (or Not provided),8,"Commerce, Management, Economics, Management",Bac+1,...,2024-08-06 12:35:07,More than once per week,Not specified (or Not provided),3.0,3.0,2024-08-01 11:53:32,Human Resources,Payroll Manager,"discover, to apply, motivate, perform",0


In [4]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd

# load model + tokenizer
model_name = "sentence-transformers/all-mpnet-base-v2"  # or any encoder model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

2025-10-24 16:10:21.704577: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-24 16:10:21.747980: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-24 16:10:21.748016: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-24 16:10:21.748047: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-24 16:10:21.756056: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-24 16:10:21.756541: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [19]:
def encode_if_not_null(text):
    if pd.notna(text) and str(text).strip():  # only if not NaN or empty
        tokens = tokenizer(str(text), return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            output = model(**tokens)
            embedding = output.last_hidden_state.mean(dim=1).squeeze().numpy()
        return embedding
    else:
        return None

df["embedding"] = df["current_role"].apply(encode_if_not_null)

In [20]:
df.head()

,binome_id,binome_statut,binome_date_proposition,binome_date_creation,binome_acceptance_delay,binome_date_update_statut,binome_cancellation_reason,binome_score,field_of_study,study_level,...,desired_exchange_frequency,hobby,project_confidence_level,project_development_level,registration_date_y,workfield,current_role,needs_to_address,status_flag,embedding
0,157657,REJECTED,2024-08-01 14:54:30,NaN,0,2024-08-03 15:00:00,Not specified (or Not provided),7,NaN,NaN,...,Not specified (or Not provided),Not specified (or Not provided),3.0,3.0,2024-08-01 00:41:51,"Accounting, management",Senior Expertise Consulting & Auditing,"discover, to apply, motivate, perform",0,"[0.108583465, 0.016112797, 0.0335505, 0.093854..."
1,157701,REJECTED,2024-08-06 15:36:55,NaN,0,2024-08-08 16:00:00,Not specified (or Not provided),12,NaN,NaN,...,Not specified (or Not provided),Not specified (or Not provided),3.0,3.0,2024-08-01 14:40:09,Banking-Finance,Manager - Financial Evaluation,"discover, to apply, perform, english",0,"[-0.120055616, 0.104448885, -0.009655574, 0.13..."
2,157710,COMPLETED,2024-08-12 15:54:34,2024-08-12 17:51:39,117,2025-05-16 16:19:05,Not applicable,11,NaN,NaN,...,Not specified (or Not provided),Not specified (or Not provided),3.0,3.0,2024-08-11 03:17:54,Computer science,Network Engineer,"discover, to apply, motivate, perform, english",1,"[0.04267117, 0.0895623, -0.1129147, -0.0662777..."
3,157715,CANCELLED,2024-08-12 16:54:52,2024-08-13 19:01:38,1566,2024-10-07 15:16:03,No response from the mentee,9,NaN,NaN,...,Not specified (or Not provided),Not specified (or Not provided),3.0,3.0,2024-08-08 15:37:28,Marketing-advertising,CRM Animation Manager,to apply,0,"[-0.10086456, 0.11101803, 0.035159186, 0.05156..."
4,157729,REJECTED,2024-08-12 17:16:43,NaN,0,2024-08-14 18:00:02,Not specified (or Not provided),8,"Commerce, Management, Economics, Management",Bac+1,...,More than once per week,Not specified (or Not provided),3.0,3.0,2024-08-01 11:53:32,Human Resources,Payroll Manager,"discover, to apply, motivate, perform",0,"[0.009633954, -0.12479902, -0.025367565, 0.053..."


In [21]:
df.to_csv("output_ecoding.csv", index=False)

In [24]:
len(df["embedding"][0])

768

In [27]:
df["embedding"].apply(type)

0        <class 'numpy.ndarray'>
1        <class 'numpy.ndarray'>
2        <class 'numpy.ndarray'>
3        <class 'numpy.ndarray'>
4        <class 'numpy.ndarray'>
                  ...           
14266         <class 'NoneType'>
14267         <class 'NoneType'>
14268    <class 'numpy.ndarray'>
14269         <class 'NoneType'>
14270         <class 'NoneType'>
Name: embedding, Length: 14271, dtype: object

In [28]:
df["embedding_workfield"] = df["workfield"].apply(encode_if_not_null)

In [29]:
df.to_csv("output_ecoding_with_workfield.csv", index=False)